In [1]:
import tensorflow as tf
import pandas as pd
import cv2
from tqdm import tqdm
import os

## Image processing

In [155]:
description = pd.read_csv('description.csv')
all_image_indexes = description.full_image_index.unique()

all_results = []
for img_index in tqdm(all_image_indexes):
    r_path = 'data/' + description[(description.full_image_index == img_index) & (description.color == 'r')].image_path.values[0]
    g_path = 'data/' + description[(description.full_image_index == img_index) & (description.color == 'g')].image_path.values[0]
    b_path = 'data/' + description[(description.full_image_index == img_index) & (description.color == 'b')].image_path.values[0]

    r_channel = cv2.imread(r_path, cv2.IMREAD_GRAYSCALE)
    g_channel = cv2.imread(g_path, cv2.IMREAD_GRAYSCALE)
    b_channel = cv2.imread(b_path, cv2.IMREAD_GRAYSCALE)
    merged_image = cv2.merge([b_channel, g_channel, r_channel])
    merged_image = cv2.resize(merged_image, (256, 256))
    cv2.imwrite(f'images/{str(img_index).zfill(5)}.jpg', merged_image)

100%|██████████| 100/100 [00:04<00:00, 21.95it/s]


In [156]:
train_df = pd.read_csv('train.csv', header=None)
train_df.columns = ['image', 'type']
train_df = pd.get_dummies(train_df, columns=['type'])
train_df

,image,type_Off,type_On,type_no_cars
0,00001.jpg,0,0,1
1,00002.jpg,1,0,0
2,00003.jpg,1,0,0
3,00004.jpg,0,1,0
4,00005.jpg,1,0,0
...,...,...,...,...
95,00096.jpg,0,0,1
96,00097.jpg,0,0,1
97,00098.jpg,0,1,0
98,00099.jpg,1,0,0


In [157]:
images = []
labels = []

for row in train_df.iloc:
  image = cv2.imread('images/' + row.image)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  images.append(image)
  labels.append([row.type_Off, row.type_On, row.type_no_cars])

In [158]:
len(images), len(labels)

(100, 100)

In [159]:
dataset = tf.data.Dataset.from_tensor_slices((images, labels))

In [160]:
dataset = dataset.batch(10).shuffle(10)

In [161]:
len(dataset)

10

In [162]:
train_dataset = dataset.take(9)
validation_dataset = dataset.skip(9)

## Define model

In [169]:
inputs = tf.keras.Input(shape=(256, 256, 3))

augment_layers = tf.keras.layers.RandomFlip("horizontal_and_vertical")(inputs)
augment_layers = tf.keras.layers.RandomRotation(0.2)(augment_layers)
augment_layers = tf.keras.layers.RandomContrast(0.1)(augment_layers)

x = tf.keras.layers.Conv2D(32, 3, strides=1, padding='same', activation='relu')(augment_layers)
x = tf.keras.layers.MaxPool2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.6)(x)

x = tf.keras.layers.Conv2D(64, 3, strides=2, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.4)(x)

x = tf.keras.layers.Conv2D(128, 3, strides=2, padding='same', activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Conv2D(256, 3, strides=1, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Conv2D(512, 3, strides=1, padding='same', activation='relu')(x)
x = tf.keras.layers.MaxPool2D((2, 2), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)

x = tf.keras.layers.Dense(256, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)

x = tf.keras.layers.Dense(3)(x)

model = tf.keras.Model(inputs=inputs, outputs=x, name='VezdekodCV50Model')
model.summary()

Model: "VezdekodCV50Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 random_flip_20 (RandomFlip)  (None, 256, 256, 3)      0         
                                                                 
 random_rotation_20 (RandomR  (None, 256, 256, 3)      0         
 otation)                                                        
                                                                 
 random_contrast_20 (RandomC  (None, 256, 256, 3)      0         
 ontrast)                                                        
                                                                 
 conv2d_86 (Conv2D)          (None, 256, 256, 32)      896       
                                                                 
 max_pooling2d_80 (MaxPoolin  (None, 128, 128, 32

In [170]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)

In [171]:
EPOCHS = 250
history = model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=EPOCHS)

Epoch 1/250
9/9 [==============================] - 3s 63ms/step - loss: 1.4700 - accuracy: 0.3222 - val_loss: 1.3912 - val_accuracy: 0.4000
Epoch 2/250
9/9 [==============================] - 0s 33ms/step - loss: 1.4129 - accuracy: 0.3667 - val_loss: 1.2897 - val_accuracy: 0.5000
Epoch 3/250
9/9 [==============================] - 0s 33ms/step - loss: 1.3267 - accuracy: 0.5111 - val_loss: 1.3508 - val_accuracy: 0.4000
Epoch 4/250
9/9 [==============================] - 0s 33ms/step - loss: 1.3413 - accuracy: 0.5000 - val_loss: 1.2599 - val_accuracy: 0.6000
Epoch 5/250
9/9 [==============================] - 0s 33ms/step - loss: 1.2923 - accuracy: 0.5333 - val_loss: 1.3172 - val_accuracy: 0.4000
Epoch 6/250
9/9 [==============================] - 0s 34ms/step - loss: 1.2830 - accuracy: 0.5667 - val_loss: 1.6214 - val_accuracy: 0.3000
Epoch 7/250
9/9 [==============================] - 0s 33ms/step - loss: 1.2712 - accuracy: 0.5444 - val_loss: 1.3317 - val_accuracy: 0.4000
Epoch 8/250
9/9 [===

KeyboardInterrupt: ignored

In [ ]:
import numpy as np
image_for_test = '00100.jpg'
image = cv2.imread('images/' + image_for_test, cv2.IMREAD_COLOR)
image = np.expand_dims(image, axis=0)
model.predict(image)

In [153]:
model.save('VezdekodCV50Model.h5')

In [154]:
model.save_weights('weights.h5')

In [173]:
model = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False, weights='imagenet', input_shape=(256, 256, 3))

In [174]:
model.trainable = False

In [179]:
inputs = tf.keras.Input(shape=(256, 256, 3))
augment_layers = tf.keras.layers.RandomFlip("horizontal_and_vertical")(inputs)
x = model(augment_layers, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)
x = tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))(x)
x = tf.keras.layers.Dense(256, activation='sigmoid')(x)
predictions = tf.keras.layers.Dense(3, activation='softmax')(x)
new_model = tf.keras.Model(inputs=inputs, outputs=predictions)
new_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_26 (InputLayer)       [(None, 256, 256, 3)]     0         
                                                                 
 random_flip_22 (RandomFlip)  (None, 256, 256, 3)      0         
                                                                 
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 global_average_pooling2d_23  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_92 (Dropout)        (None, 2048)              0         
                                                                 
 batch_normalization_280 (Ba  (None, 2048)             8192      
 tchNormalization)                                         

In [183]:
new_model.compile(
  optimizer=tf.keras.optimizers.Adam(lr=0.0001),
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
  metrics=['accuracy']
)

EPOCHS = 100
history = new_model.fit(train_dataset,
                    validation_data=validation_dataset,
                    epochs=EPOCHS)

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


9/9 [==============================] - 7s 210ms/step - loss: 0.5007 - accuracy: 0.9778 - val_loss: 0.4889 - val_accuracy: 1.0000
Epoch 2/100
9/9 [==============================] - 1s 60ms/step - loss: 0.4470 - accuracy: 0.9778 - val_loss: 0.5027 - val_accuracy: 0.9000
Epoch 3/100
9/9 [==============================] - 0s 50ms/step - loss: 0.4221 - accuracy: 1.0000 - val_loss: 0.4463 - val_accuracy: 1.0000
Epoch 4/100
9/9 [==============================] - 0s 51ms/step - loss: 0.4286 - accuracy: 0.9889 - val_loss: 0.4651 - val_accuracy: 1.0000
Epoch 5/100
9/9 [==============================] - 0s 50ms/step - loss: 0.4074 - accuracy: 1.0000 - val_loss: 0.6156 - val_accuracy: 0.9000
Epoch 6/100
9/9 [==============================] - 0s 50ms/step - loss: 0.4109 - accuracy: 1.0000 - val_loss: 0.4001 - val_accuracy: 1.0000
Epoch 7/100
9/9 [==============================] - 0s 51ms/step - loss: 0.4086 - accuracy: 0.9889 - val_loss: 0.3979 - val_accuracy: 1.0000
Epoch 8/100
9/9 [==============

In [186]:
image_for_test = '00100.jpg'
image = cv2.imread('images/' + image_for_test, cv2.IMREAD_COLOR)
image = np.expand_dims(image, axis=0)
new_model.predict(image)

array([[0.01006266, 0.00456379, 0.9853735 ]], dtype=float32)

In [187]:
new_model.save('VezdekodCV50ModelInceptionV3.h5')

In [188]:
new_model.save_weights('weightsInceptionV3.h5')